In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math
from math import log

import pprint

- ID3(based on information gain)
- C4.5(based on information gain ratio)
- CART(gini index)

#### entropy: $H(x) = -\sum_{i=1}^{n}p_i\log{p_i}$
#### conditional entropy: $H(X|Y)=\sum_$

In [27]:
def create_data():
    dataSet = [[1, 1, 'yes'],
           [1, 1, 'yes'],
           [1, 0, 'no'],
           [0, 1, 'no'],
           [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels

In [28]:
datasets, labels = create_data()

In [4]:
train_data = pd.DataFrame(datasets, columns=labels)

In [5]:
train_data

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [10]:
def calc_shannon_ent(dataset):
    num_entries = len(dataset)
    label_count = {}
    for feat in dataset:
        current_label = feat[-1]
        if current_label not in label_count:
            label_count[current_label] = 0
        label_count[current_label] +=1
    shannon_ent = 0
    for num in label_count.values():
        prob_i = num/num_entries
        shannon_ent -= prob_i * log(prob_i, 2)
    return shannon_ent



In [11]:
calc_shannon_ent(datasets)

0.9709505944546686

In [25]:
def split_dataset(dataset, axis, value):
    ret_dataset = []
    for feat in dataset:
        if feat[axis] == value:
            reduced_feat = feat[:axis]
            reduced_feat.extend(feat[axis+1:])
            ret_dataset.append(reduced_feat)
    return ret_dataset

def choose_best_feat_to_split(dataset):
    num_feat = len(dataset[0])-1
    base_ent = calc_shannon_ent(dataset)
    best_info_gain = 0
    best_feat = -1
    for i in range(num_feat):
        feat_list = [example[-1] for example in dataset]
        unique_values = set(feat_list)
        new_ent = 0
        for value in unique_values:
            subdataset = split_dataset(dataset, i, value)
            prob_i = len(subdataset)/len(dataset)
            new_ent += prob_i * calc_shannon_ent(subdataset)
        info_gain = base_ent - new_ent
        
        if(info_gain > best_info_gain):
            best_info_gain = info_gain
            best_feat = i
    return best_feat
            

In [13]:
def majority_cnt(class_list):
    class_count = {}
    for vote in class_list:
        if vote not in class_count.keys():
            class_count[vote] = 0
        class_count[vote] += 1
    sorted_class_count = sorted(class_count.items(), key=lambda item:item[1])
    return sorted_class_count[0][0]

In [23]:
def create_tree(dataset, labels):
    class_list = [example[-1] for example in dataset]
    if class_list.count(class_list[0])==len(class_list):
        return class_list[0]
    if len(dataset[0])==1:
        return majority_cnt(class_list)
    best_feat = choose_best_feat_to_split(dataset)
    best_feat_label = labels[best_feat]
    my_tree = {best_feat_label:{}}
    del (labels[best_feat])
    feat_values = [example[best_feat] for example in dataset]
    unique_value = set(feat_values)
    for value in unique_value:
        sublabels = labels[:]
        my_tree[best_feat_label][value] = create_tree(split_dataset(dataset, best_feat, value), sublabels)
    return my_tree

In [29]:
mytree = create_tree(datasets, labels)
mytree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}